In [1]:
import torch

In [2]:
# Initialize the parameters here...
ENCODER_NUM = 1
B = 256
L = 256
D_ENCODER = 768
D_SIT = 1024
PROJ = torch.nn.Linear(D_SIT, D_ENCODER)

def mean_flat(x):
    return torch.mean(x, dim=list(range(1, len(x.size()))))

zs = []  # <-- features from encoders
zs_tilde = []  # <-- projected SiT features

for _ in range(ENCODER_NUM):
    encoder_feat = torch.randn(B, L, D_ENCODER)
    zs.append(encoder_feat)

for _ in range(ENCODER_NUM):
    sit_feat = torch.randn(B, L, D_SIT)
    zs_tilde.append(PROJ(sit_feat))

print(zs_tilde[0].shape)
print(zs[0].shape)

torch.Size([256, 256, 768])
torch.Size([256, 256, 768])


In [10]:
### Check if the original augmentation loss is working...
proj_loss = 0.
bsz = zs[0].shape[0]
for i, (z, z_tilde) in enumerate(zip(zs, zs_tilde)):
    # B x L x D
    for j, (z_j, z_tilde_j) in enumerate(zip(z, z_tilde)):
        # L x D
        z_tilde_j = torch.nn.functional.normalize(z_tilde_j, dim=-1) 
        z_j = torch.nn.functional.normalize(z_j, dim=-1)
        proj_loss += mean_flat(-(z_j * z_tilde_j).sum(dim=-1))
print(f"Summed loss: {proj_loss}")
proj_loss /= (len(zs) * bsz)
print(f"Averaged loss: {proj_loss}")

Summed loss: 0.02978396601974964
Averaged loss: 0.00011634361726464704


In [ ]:
### Let's investigate how we can make the semantic relation loss(regularization) work...

### We can do it on three dimensions (B, L, D), see which one works
### For the L dimension, we can drop drop the MLP projection layer since the D dimension is canceled out

